# <span style="color:cornflowerblue"> Gerald Jones, Kimon Swanson, Alex Hines</span>
# <span style="color:cornflowerblue"> Home Work 5: Farmers Problem</span>
# <span style="color:cornflowerblue"> ISE522 Spg 22</span>

## Notebook Links:
1. [Data Display section](#Data-Display)
2. [Method Definitions](#Method-Definitions)
3. [Model Formulation Problem 1](#Model-Formulation-1)
4. [Gurobi Implementation Problem 1](#prob1)
5. [Solution Discussion Problem 1](#solution1)
6. [Model Formulation Problem 2](#Model-Formulation-2)
7. [Gurobi Implementation Problem 2](#prob2)
8. [Solution Discussion Problem 2](#solution2)
9. [Model Formulation Problem 3](#Model-Formulation-3)
10. [Gurobi Implementation Problem 3](#prob3)
11. [Solution Discussion Problem 3](#solution3)

## Problem Description:
> The farmer's problem Consider and farmer in east Tennessee who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop. The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler. Any production in excess of the feeding requirement would be sold. Over the last decade, mean selling prices have been $\text{\$170}$ and $\text{\$150}$ per ton of wheat and corn, respectively. The purchase prices are 40% more than this due to the wholesaler's margin and transportation costs.
<br><p>
> Another profitable crop is **sugar beet**, which he expects to sell at $\text{\$36}$/T; however, the EU imposes a quota on sugar beet production. Any amount in excess of the quota can be sold onlv at $\text{\$10}$/T. The farmer'squota for next year is 6,000 T. Based on past experience, the farmer knows that he mean yield on his land is roughly 2.5 T,3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. The table on the next slide summarizes these data and provides the planting costs for these crops. Formulate a linear program to maximize total profit (planting cost —revenue from sales).</p>

# Extentions To Accomplish:
1. [Suppose the farmer allocates 120 acres to wheat, 80 to corn, and 300 for sugar beets. But also suppose that the 
   yields are not certain but are subject to the same 3 scenarios we discussed in class (above average, average, below 
   average, each with equation probability. What is the expected annual profit?](#prob1)
    - [Problem 1 Method Defintion and Execution Cell](#prob1-meth)
2. [Consider an extension of the problem in which prices also are uncertain. Specifically, when yields are above 
   average prices for corn and wheat go down by 10% and when yields are below average, they go up by 10%.
   Assume sugar beet prices are not affected by yields. Formulate and solve a stochastic linear program and solve 
   it with a solver.](#prob2)
3. Consider a different version of the problem in which the farmer possesses four fields of sizes 185, 145, 105, and 
   65, respectively. The total of 500 acres is unchanged. However, the farmer wishes to only plant one type of crop 
   on each field. Formulate and solve a stochastic program and solve it with a solver

## Notes and Observations
* Total of 500 acres

* 3 possible crops:
    - wheat 
        + (>= 200 T through crops or purchase)
        + $\text{\$170}$/T selling price
        + $\text{\$170}$*40  + $\text{\$170}$ purchase price = $\text{\$238}$/T
        + AVG yield: 2.5 T
    - corn  
        + (>= 240 T through crops or purchase)
        + $\text{\$150}$/T selling price
        + $\text{\$150}$*40 + $\text{\$150}$ purchase price = $\text{\$210}$/T
        + AVG yield: 3 T
    - sugar beats
        + selling price is tiered
            - below quota:= $\text{\$36}$/T
            - above quota:= $\text{\$10}$/T
        + quota:= 6000 T
        + AVG yield: 20 T

* any purchase of wheat or corn over the required amount is sold for profit at indicated selling price

## Assumptions:


# <span style="color:orange"><center><b>Module imports and data loading</b></center></span>

In [2]:
from _GUROBI_TOOLS_.GUROBI_MODEL_BUILDING_TOOLS import *
from _NOTE_BOOK_UTILS import *
import numpy as np


<IPython.core.display.Javascript object>

# <a id=Data-Display><span style="color:Green"><center> Data Display & Python Variable Assignments</center></span></a>

In [6]:
# display data for problem
# data is represented by the instantiated dataframe show here and displayed in the output below

# data for the problem
data_df = pd.DataFrame(
{
    "Wheat":[2.5, 150, 170, 170, 238, 200], 
    "Corn": [3,   230, 150, 150, 210, 240],
    "Sugar Beets":[20, 260, 36, 10, 0, 0],
},
    index=["Yield", "Planting Cost", "Selling Cost1", "Selling Cost2", "Price", "min"]
)

display(data_df)

# used to calculate the above/below average yields
average_rate = .2


,Wheat,Corn,Sugar Beets
Yield,2.5,3,20
Planting Cost,150.0,230,260
Selling Cost1,170.0,150,36
Selling Cost2,170.0,150,10
Price,238.0,210,0
min,200.0,240,0


# <a id=Method-definitions><span style="color:red"><center>Method Definitions</center></span></a>

# <a id=prob1-meth>Problem 1 Solver method</a>
[Problem 1 Method Defintion and Execution Cell](#prob1-meth) | [Problem 1 Solution](#prob1)

In [77]:
# calculate annual profit with yield uncertainty
def objective_uncertain_yields(df, xw, ysc, quota, minT_dict, scenerio_probs=None, 
                               equally_likely=True, ):
    """
            df: dataframe containing columns Wheat, Corn, Sugar Beets, and indices Planting Cost, Yield, Selling Cost1, 
                and Selling Cost2. The Yield,"Crop" is the expected yield rate of the given crop amount xw
            xw: list representing the amount of each crop that was planted. takes the form:
                    xw[0] --> amount of Wheat planted
                    xw[1] --> amount of Corn planted  
                    xw[2] --> amount of Sugar Beets planted  
            ysc: list of lists for the expected % of planted crop yielded. Takes the form:
                    ysc[scenerio, crop]
                    crop:  {0 (wheat), 1 (corn), 2 (Sugar Beets)}
                    scenerio: {0 (below average percentage of planted yielded), 
                               1 (just 1 representing the expected average yield), 
                               2 (above average percentage of planted yielded)} 
    """
    #              ***** calculate and return annual costs under uncertainty *****
    
    
    if scenerio_probs is None:
        scenerio_probs = list()
        num_cases = len(ysc.shape[1])
        for i in range(num_cases):
            scenerio_probs.append(1.0/num_cases)
    
    # amount of each crop planted
    wheat_planted = xw[0]
    corn_planted = xw[1]
    sugarbeets_planted = xw[2] 
    
    # average expected yields
    avg_wheat = df.loc["Yield", "Wheat"]
    avg_corn = df.loc["Yield", "Corn"]
    avg_sugarbeets = df.loc["Yield", "Sugar Beets"]
    
    # costs for planting the given acreages of each type of crop
    wheat_costs = df.loc["Planting Cost", "Wheat"]*wheat_planted
    corn_costs = df.loc["Planting Cost", "Corn"]*corn_planted
    sugarbeets_costs = df.loc["Planting Cost", "Sugar Beets"]*sugarbeets_planted
    
    # Total planting costs
    planting_costs = wheat_costs + corn_costs + sugarbeets_costs
    print("\t\t\t---------------Planting Costs for Given Crop Allotments---------------")
    print("\tWheat planting costs: {:>22.0f}".format(wheat_costs))
    print("\tCorn planting costs: {:>23.0f}".format(corn_costs))
    print("\tSugar beets planting costs: {:>16.0f}".format(sugarbeets_costs))
    print("\tTotal Planting Costs: {:>23.0f}".format(planting_costs))
    print("------------------------------------------------------------------------\n")
    
    # yield amounts for each scenerio
    # (below average= average + average*-below_average_rate = average(1 - rate))
    # (average = average + average*0 = average*(1 + 0)
    # (above average= average + average*above_average_rate = average(1 + rate))-->
    # make list for each crop where [below_average_yield, average_yield, above_average_yield]
    wheat_yields = [avg_wheat*(1+ysc[0, scenerio])*xw[0] for scenerio in range(len(ysc))]
    corn_yields = [avg_corn*(1+ysc[1, scenerio])*xw[1] for scenerio in range(len(ysc))]
    sugarbeets_yields = [avg_sugarbeets*(1+ysc[2, scenerio])*xw[2] for scenerio in range(len(ysc))]
    print("\t\t\t---------------Expected Yields (tons) for Each Possible Case---------------")
    print("\nWheat yields:\n\tBelow Average {:.2f},\n\tAverage {:.2f},\n\tAbove Average {:.2f}".format(wheat_yields[0], 
                                                                                        wheat_yields[1],
                                                                                        wheat_yields[2]))
    
    print("\nCorn yields:\n\tBelow Average {:.2f},\n\tAverage {:.2f},\n\tAbove Average {:.2f}".format(corn_yields[0], 
                                                                                     corn_yields[1],
                                                                                     corn_yields[2]))
    print("\nSugarbeet yields:\n\tBelow Average {:.2f},\n\tAverage {:.2f},\n\tAbove Average {:.2f}".format(sugarbeets_yields[0], 
                                                                                             sugarbeets_yields[1],
                                                                                             sugarbeets_yields[2]))
    print("------------------------------------------------------------------------\n")
#     wheat_profits = df.loc["Yield", "Wheat"]*((1-ysc[0, 0]) + ysc[1, 0] + (1+ysc[2, 0]))*xw[0]*df.loc["Selling Cost1", "Wheat"]
#     corn_profits = df.loc["Yield", "Corn"]*((1-ysc[0, 1]) + ysc[1, 1] + (1+ysc[2, 1]))*xw[1]*df.loc["Selling Cost1", "Corn"]
    
    # profits based on different scenerios
    # excess_wheat = a if a < b else b
    # use yields to: 
    #         1) calculate the amount of wheat/corn that needs to be purchased
    #         2) calculate any excess wheat/corn that can be sold for profit
    
    #                 1) Amount to purchase for each scenerio
    excess_wheat, missing_wheat = list(), list()
    excess_corn, missing_corn = list(), list()
    excess_sugarbeets, justright_sugarbeets = list(), list() 
    
    
    for wheatSi, cornSi, sugarbeetSi in zip(wheat_yields, corn_yields, sugarbeets_yields):
        # calculate the excess for scenerio i(below, average, above)
        excess_wheat.append(wheatSi - minT_dict["Wheat"] if wheatSi - minT_dict["Wheat"] >= 0 else 0 )
        missing_wheat.append(minT_dict["Wheat"] - wheatSi if minT_dict["Wheat"] - wheatSi >= 0 else 0)
        
        # 
        excess_corn.append(cornSi - minT_dict["Corn"] if cornSi - minT_dict["Corn"] >= 0 else 0 )
        missing_corn.append(minT_dict["Corn"] - cornSi if minT_dict["Corn"] - cornSi >= 0 else 0)
    
        # 
        excess_sugarbeets.append(sugarbeetSi - quota  if sugarbeetSi > quota else 0)
        justright_sugarbeets.append(quota  if sugarbeetSi > quota else sugarbeetSi)
    
    
    # Profits:
    wheat_profits = (np.array(excess_wheat) * df.loc["Selling Cost1", "Wheat"]).flatten()
    corn_profits = (np.array(excess_corn) *  df.loc["Selling Cost1", "Corn"]).flatten()
    sugarbeets_profits = np.array(excess_sugarbeets) * df.loc["Selling Cost2", "Sugar Beets"] + np.array(justright_sugarbeets) * df.loc["Selling Cost1", "Sugar Beets"]
    
    # Purchase Costs:
    print("\t\t\t---------------Expected Excess Crops for each case---------------")
    wm_fl = np.array(missing_wheat).flatten()
    we_fl = np.array(excess_wheat).flatten()
    cm_fl = np.array(missing_corn).flatten()
    ce_fl = np.array(excess_corn).flatten()
    
    print("Excess Wheat:\n\tBelow Average{:>23.0f},\n\tAverage{:>23.0f},\n\tAbove Average{:>23.0f}".format(
                                                                                we_fl[0], 
                                                                                we_fl[1], 
                                                                                we_fl[2]))
    print("Excess Corn:\n\tBelow Average{:>23.0f},\n\tAverage{:>23.0f},\n\tAbove Average{:>23.0f}".format(
                                                                                ce_fl[0], 
                                                                                ce_fl[1], 
                                                                                ce_fl[2]))
    
    print("------------------------------------------------------------------------\n")
    
    print("\t\t\t---------------Expected Crops That need to be purchased for each case---------------")
    print("\nWheat missing:\n\tBelow Average{:>23.0f},\n\tAverage{:>23.0f},\n\tAbove Average{:>23.0f}".format(
                                                                                wm_fl[0], 
                                                                                wm_fl[1], 
                                                                                wm_fl[2]))
    print("\nCorn missing:\n\tBelow Average{:>23.0f},\n\tAverage{:>23.0f},\n\tAbove Average{:>23.0f}".format(
                                                                                cm_fl[0], 
                                                                                cm_fl[1], 
                                                                                cm_fl[2]))
    

    
    # calculate the cost of purchasing needed wheat and corn
    wheat_purhased_costs = np.array(missing_wheat) * df.loc["Price", "Wheat"]
    corn_purhased_costs = np.array(missing_corn).flatten() * df.loc["Price", "Corn"]
    print("\t\t\t---------------Purchase Costs---------------")
    print("wheat purchased: ", wheat_purhased_costs)
    print("Corn purchased: ", corn_purhased_costs)
    print("------------------------------------------------------------------------\n")
    
    
    
    
    # Total purchasing costs
    purchasing_costs = wheat_purhased_costs + corn_purhased_costs
    
    print()
    print("\t\t\t---------------Sales profits---------------")
    print("wheat profits: ", wheat_profits)
    print("corn profits: ", np.array(corn_profits).flatten())
    # sugar beets to sell at the different levels of sales cost
    print("Sugar beets within quota: ", justright_sugarbeets)
    print("Sugar beets above quota: ", excess_sugarbeets)
    print("Sugar beets profits: ", sugarbeets_profits)
    # total profits from sales
    sale_profits = wheat_profits + corn_profits + sugarbeets_profits
    print("Sales Profits Total: ", sale_profits)
    print("------------------------------------------------------------------------\n")
    
    print("\t\t\t---------------Total Costs and Profits---------------")
    print("Sales Profits Total: ", sale_profits)
    print("Planting Costs: ", planting_costs)
    print("Purchasing Costs: ", purchasing_costs)
    print("------------------------------------------------------------------------\n")
    
    b_avg_obj = sale_profits[0] - planting_costs - purchasing_costs[0]
    avg_obj = sale_profits[1] - planting_costs - purchasing_costs[1]
    a_avg_obj = sale_profits[2] - planting_costs - purchasing_costs[2]
    print()
    print("\t\t\t---------------Expected Profits for Each Yield Scenerio---------------")
    print("Below Average Expected Profits: {}".format(b_avg_obj))
    print("Average Expected Profits: {}".format(avg_obj))
    print("Above Average Expected Profits: {}".format(a_avg_obj))
    print("------------------------------------------------------------------------\n")
    
    expected_profit = scenerio_probs[0]*below_average_case +\
                        scenerio_probs[1]*average_case +\
                        scenerio_probs[2]*above_average_case
    
    return (b_avg_obj, avg_obj, a_avg_obj),  expected_profit 

#######################################################################################
# Initial amount of acreage devoted to each crop type
crop_acreage = [
    120, # acres of wheat to plant
    80,  # acres of corn to plant
    300  # acres of sugar beets to plant
]

print("Initial Allotted Acreage:")
print("Wheat: {}".format(crop_acreage[0]))
print("Corn: {}".format(crop_acreage[1]))
print("Sugar Beets: {}".format(crop_acreage[2]))

# adjustment to yield rate based on scenerio
# rows-> crops, columns->scenerio
# In this problem the scenerios are expected to see a 20% decrease, or increase 
# for the below and above average cases respectively
crop_yieldRate_scenerio = np.array([
                [-.2, 0, .2],  # below average, average, above average-->wheat
                [-.2, 0, .2],  # below average, average, above average-->corn
                [-.2, 0, .2],] # below average, average, above average-->sugar beets
              )

# Minimum amount of wheat/Corn that must be yielded for feed crop purposes
minT_dict = {
    "Wheat":200,        # minimum tons of wheat needed for feed
    "Corn":240,         # minimum tons of corn needed for feed
}

print("\t\t\t------------------------- Minimum crop resources for feed crops")
for c in minT_dict:
    print("Require a minimum of {} tons of {}".format(minT_dict[c], c))
print("-------------------------------------------------")
# print(ysc)
# print(len(ysc))
quota = 6000               # yield level that when exceeded causes price of sugar beets-->reduced/penalized

# Probabilities for he below, average, and above average cases
scenerio_probs = [
    1/3,  # below average yield probability
    1/3,  # average yield probability
    1/3,  # above average yield probability
]

# call main method that performs calculation
three_cases, expected_profit =  objective_uncertain_yields(
                                                            data_df, #(Yield, Price, Selling-Prices)/Crop
                                                            crop_acreage,    # amount of aceres devoted to each crop
                                                            crop_yieldRate_scenerio,   # yield rate adjustment/yield scenerio
                                                            quota, # sugar beets yield quota
                                                            minT_dict,  # key:=crop, val=minimum tons of crop for feed
                                                            scenerio_probs, # probability of each scenerio
                                                            equally_likely=True, 
                                                          )

# Parse below, average, and above average expected profits
below_average_case = three_cases[0]
average_case = three_cases[1]
above_average_case = three_cases[2]

# Print the calculated Expected cost based on the three cases for yield amounts
# and the given crop acreage allotments
print("Expected Profit: {}".format(expected_profit))


expected_profit = probs[0]*below_average_case + probs[1]*average_case + probs[2]*above_average_case
print("Above average case Profit: {}".format(above_average_case))
print("Average case Profit: {}".format(average_case))
print("Below average case Profit: {}".format(below_average_case))

# calculate it again just to do a sanity check
print("Expected Profit: {}".format(expected_profit))

Initial Allotted Acreage:
Wheat: 120
Corn: 80
Sugar Beets: 300
			------------------------- Minimum crop resources for feed crops
Require a minimum of 200 tons of Wheat
Require a minimum of 240 tons of Corn
-------------------------------------------------
			---------------Planting Costs for Given Crop Allotments---------------
	Wheat planting costs:                  18000
	Corn planting costs:                   18400
	Sugar beets planting costs:            78000
	Total Planting Costs:                  114400
------------------------------------------------------------------------

			---------------Expected Yields (tons) for Each Possible Case---------------

Wheat yields:
	Below Average 240.00,
	Average 300.00,
	Above Average 360.00

Corn yields:
	Below Average 192.00,
	Average 240.00,
	Above Average 288.00

Sugarbeet yields:
	Below Average 4800.00,
	Average 6000.00,
	Above Average 7200.00
------------------------------------------------------------------------

			---------------Ex

# <a id=prob1>Problem 1 Solution</a>:
[Problem 1 Method Defintion and Execution Cell](#prob1-meth)
The above code cell defines a function that calculates the expected profit when hedging on the three possible yield scenerios. The following sections will describe the three stages of the process and the resulting expected profit. 

## Stage 1: Deciding how much to plant
> For stage one the amounts to plant are given in the problem and are;
> *       wheat = 120 
> *        corn = 80
> * sugar beets = 300

## Attempting to Quantify Uncertainty into 3 scenerios:
> From the problem describe in class there are three possible scenerios of yields rates (acres/ton) that are considered. 
> 1. The yield amount is below average i.e. 20% less than average 
> 2. The yield amount is average i.e. what is found in the provided data table
> 3. The yield amount is above average i.e. 20% more than average
> Using the yield rates of 80% for below average, 100% for average, and 120% for above average each expected yield is increased by the corresponding yield case (below average, average, above average) to produce the expected yields seen in the above output labled, "Wheat yields", "Corn yields", "Sugar Beets yields" each with the corresponding yield case. These amounts can been seen in the above cells output under the "**Expected Yields (tons) for Each Possible Case**" section detailing the amount of tons of each crop is expected for each case. 

## Stage 2: How much needs to be purchased and sold 
> The amount that needs to be purchased is the difference between the amounts of feed crops we have based on expected yield and the minimum amounts needed for feed crops. If the amount of feed crops (wheat/corn) yielded is below the minimum amount required for feed then the difference must be purchased (see "Expected Crops That need to be purchased for each case" section). If for a given scenerio the amount of feed crops are above the minimum amount the difference can be sold for a profit(see "Expected Excess Crops for each case" section). For each crop there is a set purchasing price and for each of the feed crops there is a set selling price. However, for the sugar beets there is a two level system of selling prices. If the amount of sugar beets yielded is above the set quota of 6kT then there is a higher sale price for each ton below this quota. Every ton above the quota must be sold at a lower selling price. If the amount sold is at or below the quota then only the higher price need be considered. If the amount of sugar beets yielded is in excess of the quota the difference between the quota and the amount of sugar beets yielded must be sold at the lower selling price. Thus the amount that needs to be sold in each scenerio is based on how much excess wheat, and corn we have, and the amount of sugar beets we have. The amount to be purhased is the difference between the minimum wheat and corn amounts and what is yielded in a given scenerio and only matters when there are less crops yielded than the minimum. 

## Calculating the Expected Yield:

## Expected Profit
> To calculate the expected profit with the uncertainties you must calculate the expected profit in each of the scenerios and sum these expected profits weighted by their probability of occurance. For each scenerio the objective is:<br>
> <center>-planting_costs - purchasing_cost + sales_profits = gross_profit</center>
> This Calculation must be performed for each yield scenerio. To accomplish this the method uses a set of matrices where the rows represent one of the crops and the columns represent one of the scenerios for the sales profits, and purchasing costs operations. 

### Planting Costs:
> The above defined and called method calcultes the planting costs for each crop and sums them to get the total planting costs displayed in the above output under the "Planting Costs for Given Crop Allotments" section of the output. These costs are constant across the scenerios because it is assumed that the given allotsments are used for each with differing yield conditions. 

### Sales Profits & Purchasing Costs:
#### Sales Profits:
> To calculate the sales profits two calculations are performed. First the amount of excess of each crop is calculated and the profit from the sale of this amount is calculated. For the sugar beets any amount of yield is considred excess and thus will be sold, but the amount it sells for is dependent on the yield for the scenerio. If the yeild is above the quota then the first 6K tons will be sold for the higher 36 ($\text{\$}$/T) rate, and the remaining will be sold at the lower 10 ($\text{\$}$/T) rates. For this step two arrays are kept. One represents the amount in each scenerio that is within the quota. If the yield for sugar beets does not exceed the quota then it is just this amount, if the yielded is above the quota then this value is set to the quota, and the excess is calculated and stored in the second array that holds the amount over the quota for a given scenerio. In a similar manner the amount in excess for the feed crops and the amount lacking and thus needing to be purchased based on the minimum are also kept as arrays. These arrays can be seen in the ouput labled missing_wheat, missing_corn, and justright_sugarbeets, for the arrays for the below, average, and above average cases for the amounts in each that are lacking in the feed crops and or that is below the quota for the sugar beets. The outputs labeled excess_wheat, excess_corn, and excess_sugarbeets are arrays of similar form for the amounts of extra wheat, and corn above the feed crop minimum is available for each scenerio and the amount of sugar beets over the quota repsectively. To calculate the sales profits in each scenerio the sales price for each crop type is multiplied by the excess crop arrays and the justright_sugarbeets arrays with the appropriate sales prices based on the table. This produces three arrays representing the below average, average, and above average cases for each of the three crops as seen in the "Sales profits" section of the output. At the end of the section an array labeled total sales profits is displayed that represents the total sales profits for each scenerio which is just the column wise summation of the three sales profits array for the crops. 

#### Purchasing Costs:
> To calculate the purchasing costs the missing_wheat, and missing_corn arrays are multiplied by the purchasing costs for the corresponding crops. The results of this calculation can be seen in the output labeled "Purchase Costs". 

## Expected Profits with Uncertainty Solution:
> To calculate the expected profit based on all three scenerios and the given acreage assignments each scenerios planting, purchasing costs are summed with the sales profits, then each scenerios profits are multiplied by the probability and these
values are summed. The results of thes operations can be seen in the output labled "Expected Profit". From the calculation the hedged allotments would lead to an **expected profit of 107240.0.**

# <a id=prob2>Problem 2</a>:
> Consider an extension of the problem in which prices also are uncertain. Specifically, when **yields are above 
average, prices for corn and wheat go down by 10%** and when **yields are below average, they go up by 10%**.
**Assume sugar beet prices are not affected by yields**. Formulate and solve a stochastic linear program and solve 
it with a solver.

## Notes:
> In this version there are levels of crop sales prices in a similar manner to the levels of sales prices for sugar beets in the previous problems. Here, the sales price of a feed crop is based on the expected yield as shown below: 
* When (feed crop yields > average), feed_crop_yields=average(1+.2): # market surplus
    - prices go down 10%
        + prices * .9
* When (feed crop yields = average), feed_crop_yields=average
    - prices are expected average
        + prices * 1
* When (feed crop yields < average), feed_crop_yields=average(1-.2): # market shortages
    - prices go up 10%
        + prices * 1.1
        
* Each case is equally likely
* Have to account for the three possibilities 

# <a id=Model-Formulation-1><center> <span style="color:blue"> Model Formulation Problem 2</span> </center></a>
* [Paremeters and Sets](#Parameters-and-Sets-2)
* [Variables](#Variables-2)
* [Equations and Constraints](#Equations-and-Constraints-2)
* [Objective](#Objective-2)

## <a id=Parameters-and-Sets-2><span style="color:DarkBlue">Parameters and Sets 2:</span></a>

### $\textbf{C}  \quad \quad \quad \text{set of possible crops {wheat(1), corn(2), sugar-beets(3)} } c \in \textbf{C}$ 
### $S \quad \quad \quad \text{ expected level for yield {below average(1), average(2), below average(3)} where s} \in S$
### $X_c \quad \quad \quad \text{acres allocated to crop c, c}\in C$
### $Y_{s,c} \quad \quad \quad \text{ tons purchased in scenerio s for crop c}$
### $E_{s,c} \quad \quad \quad \text{ Expected yield (tons) in scenerio s for crop c}$
### $W_{s,c} \quad \quad \quad \text{ tons of crop c sold in scenerio s}$
### $P_{s,c} \quad \quad \quad \text{ purchase cost for crop c } c \in \{wheat, corn\}$
### $L_{s,c} \quad \quad \quad \text{ planting cost for crop c } c \in \{wheat, corn\}$
### $C_{s,c} \quad \quad \quad \text{ purchase price for crop c in scenerion s, c }\in \{wheat(0), corn(1)\}$
### $F_{c} \quad \quad \quad \text{ sale price for feed crop c  }$
### $B_{q} \quad \quad \quad \text{ sale price for bulk sugar beets in quota level q, q }\in \{in\_quota, outside\_quota\}$
### $M_{c} \quad \quad \quad \text{ minimum amount of feed crop c}$
### $A \quad \quad \quad\text{ maximum amount of acreage available}$
### $r_s \quad \quad \quad \text{ the purchase price for yield scenerio s}$
### $\rho_{s,c} \quad \quad \quad \text{ average expected ton/acre rate for crop c for scenerio s}$


## <a id=Equations-and-Constraints-2><span style="color:DarkBlue">Equations and Constraints:</span></a>

>### <center><span style="font-size:30px;color:red"><b>Equation </b></span></center>

# First Stage:
> ### Maximum Acreage Available: $\sum_{c}^{|C|}(X_c) \leq A$

# General Constraints

> ### Minimum feed crop constraints: $Y_{s,c} + E_{s,c} - W_{s,c} \leq M_c, \forall s,c \text{ where c } \in \text{ \{wheat(0), corn(1)\}}$

# Below Average:
> ### Price average yield
> * $C_{0,c} == P_c*(r_0) \implies E_{0,c} == E_{1,c}*(.8), \text{  c} \in \{0,1\}$
> * $E_{0,c} == E_{1,c}*(.8), \forall c$
# Average:
> ### Price average yield
> * $C_{1,c} == P_c*(r_1) \implies E_{1,c} == E_{1,c}, \text{  c} \in \{0,1\}$
> * $E_{1,c} = X_c \cdot \rho_{s,c}, \forall c$
# Above Average: 
> ### Price average yield
> * $C_{2,c} == P_c*(r_2) \implies E_{2,c} == E_{1,c}*(1.2), \text{  c} \in \{0,1\}$
> * $E_{2,c} == E_{1,c}*(1.2), \forall c$
> * $ \sum_{s}^{|S|}((\sum_{c=2}^{3}W_{s,c}) \leq E_{s,c})$
> * $ \sum_{s}^{|S|}\sum_{c=0}^{1}((W_{s,c}) \leq E_{s,c})$
> * $ W_{s,2} \leq 6000, \forall s$

# <a id=Model-Formulation-1><center> <span style="color:blue"> Model Formulation Problem 3</span> </center></a>
* [Paremeters and Sets](#Parameters-and-Sets-3)
* [Variables](#Variables-3)
* [Equations and Constraints](#Equations-and-Constraints-3)
* [Objective](#Objective-3)

# <a id=Model-Formulation-1><center> <span style="color:blue"> Model Formulation Problem 2</span> </center></a>
* [Paremeters and Sets](#Parameters-and-Sets-1)
* [Variables](#Variables-1)
* [Equations and Constraints](#Equations-and-Constraints-2)
* [Objective](#Objective-1)

## <a id=Parameters-and-Sets-1><span style="color:DarkBlue">Parameters and Sets:</span></a>

### $\textbf{C}  \quad \quad \quad \text{set of possible crops {wheat(1), corn(2), sugar-beets(3)} } c \in \textbf{C}$ 
### $X_w \quad \quad \quad \text{acres allocated to wheat}$
### $X_c \quad \quad \quad \text{acres allocated to corn}$
### $X_s \quad \quad \quad \text{acres allocated to sugar beets}$
### $S \quad \quad \quad \text{ expected level for yield {below average(1), average(2), below average(3)} where s} \in S$

## <a id=Variables><span style="color:DarkBlue">Variables:</span></a>

### $X_{c} \quad  \text{     amount of acres to devote to crop c } \in C$ 
### $Y_{c,s}   \quad  \text{  tons of crop type c purchased under condition s} \subset \text{{wheat(1), corn(2)}}$
### $W_{c,s}   \quad  \text{T's of crop type c sold under condition s} \subset \text{{wheat(1), corn(2), sugarbeets @ 36 (3), sugarbeets @ 10(4)}}$
### $A   \quad \text{ maximum acres of land available}$
### $M_c\quad \text{ minimum T of crop c required for feed}$
### $M_c\quad \text{ minimum T of crop c required for feed}$


>### <center><span style="font-size:30px;color:red"><b>Equation </b></span></center>

>### <center><span style="font-size:30px;color:red"><b>Acreage Constraint </b></span></center>

# $$\sum_{c=1}^{|C|}(X_c) \leq (A=500)$$

>### <center><span style="font-size:30px;color:red"><b>Equation </b></span></center>

# $$0 \leq \quad \sum_{c=1}^{|C|}X_{w,c} \quad  \leq H_{w}, \forall w$$

## <a id=Objective><span style="color:green">Objective: </span></a>

# $$\min(T = N + M)$$

# <a id=prob1><center>Problem 1</center></a>
1. [Data Display section](#Data-Display)
2. [Method Definitions](#Method-Definitions)

# <a id=prob1><center>Gurobi Implementation and Solution Problem 1</center></a>

In [ ]:
try:
    # instantiate model object 
    m = gp.Model("Farmer_problem1")
 
    
    
    #########################################################################################
    ################################## Parameters set up ####################################
    #########################################################################################

    #########################################################################################
    ################################## Variables set up #####################################
    #########################################################################################

    
    #########################################################################################
    ################################## Objective set up #####################################
    #########################################################################################    
    
    #########################################################################################
    ################################## Constraint set up ####################################
    #########################################################################################
    
    
    #########################################################################################
    ################################## SOLVE:OPTIMIZE #######################################
    #########################################################################################    
    
    
    m.optimize()
    
    #########################################################################################
    ################################## Display Results ######################################
    #########################################################################################    
    displayDecisionVars(m, end_sentinel="6")
    
    print("\n-------------Does it make sense?----------------------")  
    print('Obj: {:.2f}'.format(m.ObjVal))
    
    
# catch some math errors
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

# <a id=prob2><center>Gurobi Implementation and Solution Problem 2</center></a>

In [ ]:
try:
    # instantiate model object 
    m = gp.Model("G_MOD")
 
    
    
    #########################################################################################
    ################################## Parameters set up ####################################
    #########################################################################################

    #########################################################################################
    ################################## Variables set up #####################################
    #########################################################################################

    
    #########################################################################################
    ################################## Objective set up #####################################
    #########################################################################################    
    
    #########################################################################################
    ################################## Constraint set up ####################################
    #########################################################################################
    
    
    #########################################################################################
    ################################## SOLVE:OPTIMIZE #######################################
    #########################################################################################    
    
    
    m.optimize()
    
    #########################################################################################
    ################################## Display Results ######################################
    #########################################################################################    
    displayDecisionVars(m, end_sentinel="6")
    
    print("\n-------------Does it make sense?----------------------")  
    print('Obj: {:.2f}'.format(m.ObjVal))
    
    
# catch some math errors
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

# <a id=prob3><center>Gurobi Implementation and Solution Problem 3</center></a>

In [ ]:
try:
    # instantiate model object 
    m = gp.Model("G_MOD")
 
    
    
    #########################################################################################
    ################################## Parameters set up ####################################
    #########################################################################################

    #########################################################################################
    ################################## Variables set up #####################################
    #########################################################################################

    
    #########################################################################################
    ################################## Objective set up #####################################
    #########################################################################################    
    
    #########################################################################################
    ################################## Constraint set up ####################################
    #########################################################################################
    
    
    #########################################################################################
    ################################## SOLVE:OPTIMIZE #######################################
    #########################################################################################    
    
    
    m.optimize()
    
    #########################################################################################
    ################################## Display Results ######################################
    #########################################################################################    
    displayDecisionVars(m, end_sentinel="6")
    
    print("\n-------------Does it make sense?----------------------")  
    print('Obj: {:.2f}'.format(m.ObjVal))
    
    
# catch some math errors
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

# <a id=solution><span style="color:crimson"><center>Solution Discussion</center></a>

> The solution....

In [ ]:
# save the notebook as a pdf
# to_PDF("notebook_title")